<center><h1>POD-Mini: minimized proof of delivery</h1></center>
<center><h2>implemented via Python</h2><center>

# I Intro

This is a Python implementaton of Pod-Mini of [zkPod](https://github.com/sec-bit/zkPoD-node) via klefki, for more details, just check the [technial Paper](https://github.com/sec-bit/zkPoD-node).

In [340]:
from klefki.types.algebra.concrete import (
    EllipticCurveGroupSecp256k1 as ECG,
    EllipticCurveCyclicSubgroupSecp256k1 as CG,
    FiniteFieldSecp256k1 as F,
    FiniteFieldCyclicSecp256k1 as CF
)


G = CG.G

In [341]:
import random

N = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
random_f = lambda: CF(random.randint(1, N) % CF.P)

q = random_f()
H = G @ q
    

In [342]:
from hashlib import sha256

hash = lambda x, y: CF(int(sha256(str(x.value + y).encode()).hexdigest(), 16) % N)

# II POD-Mini

### Initializer Phase

In [343]:
data = 123456789
tag = 6

In [344]:
m = CF(data)
o = CF(tag)

In [345]:
sigma = G @ m + H @ o

### Deliver Phase

In [346]:
k_w = random_f()

k = hash(k_w, 1)
k_ = hash(k_w, 2)
k0 = hash(k_w, 3)
k0_ = hash(k_w, 4)

In [347]:
K = G @ k + H @ k_
K0 = G @ k0 + H @ k0_
(K, K0)

(EllipticCurveGroupSecp256k1::(FiniteFieldSecp256k1::66015531106320307125058613549432177165979816616746175540199007061665771892002, FiniteFieldSecp256k1::72588995951954668146712499125332144761512332639315434212655577460971830124987),
 EllipticCurveGroupSecp256k1::(FiniteFieldSecp256k1::22835421699728706799348159755607798103766612846929226115665126667661481300161, FiniteFieldSecp256k1::55563197045843542150978957972746147264362062947215050380825114339114496509031))

* $S \rightarrow R$: $(K, K_0)$

In [348]:
c = random_f()

* $R \rightarrow S$: $c$

In [349]:
m_ = k + c * m
o_ = k_ + c * o
z = k0 + c * k
z_ = k0_ + c * k_
(m_, o_, z, z_)

(FiniteFieldCyclicSecp256k1::9515444680583591617777780486233531783412547066963278936446637506394489520388,
 FiniteFieldCyclicSecp256k1::29271541237009020691999333302951353450315171725274139455902645920816009127865,
 FiniteFieldCyclicSecp256k1::94912546539957581488528138597424537003974683205981630749008449152416262105979,
 FiniteFieldCyclicSecp256k1::12893824573842345613282480680167393126591475759351213269212348140049986424519)

* $S \rightarrow R: (\bar{m}, \bar{o}, z, z')$

R should verify that:

$$
Com(m; o)^c \cdot Com(k_0;k'_0) \stackrel{?}{=} Com(\bar{m}; \bar{o})\\
Com(k_0;k'_0) \cdot Com(k;k')^c \stackrel{?}{=} Com(z;z')
$$

In [350]:
assert sigma @ c + K == G @ m_ + H @ o_

In [351]:
assert K0 + (K @ c) == G @ z + H @ z_

### Reveal-phase

* $R \rightarrow J: \rho$
* $S \rightarrow J: k_{\omega}$

$$
z \stackrel{?}{=} H(k_{\omega}, 3) + c \cdot H(k_{\omega}, 1)
$$

In [352]:
assert z == hash(k_w, 3) + c @ hash(k_w, 1)

In [353]:
assert m == (m_ - hash(k_w, 1)) / c

In [354]:
(m_ - hash(k_w, 1)) / c

FiniteFieldCyclicSecp256k1::123456789

# POD-AS

In [355]:
from IPython.display import Image

In [356]:
f = Image("lena.jpg")

### Init Phase

In [357]:
import numpy as np

In [358]:
M = np.array(list(map(CF, f.data))).reshape(-1, 160)

In [359]:
n, s = M.shape

In [360]:
Pad = np.matrix([random_f() for _ in range(0, n)])

In [361]:
Pm = np.concatenate((Pad.T, M), axis=1).tolist()

In [362]:
U = [G @ random_f() for _ in range(0, s + 1)]

In [363]:
from functools import reduce 

def vgm(a: [CF]) -> [ECG]:
    return reduce(lambda x,y: x+y,
                  list(map(lambda a: a[0] @ a[1], zip(U, a))))

def Cv(r:CF, a:[CF]):
    return (H@r + vgm(a))

In [364]:
list(zip(U, Pm[0]))[0]

(EllipticCurveGroupSecp256k1::(FiniteFieldSecp256k1::89873864151582986093691155183877263323500274185533681427305924574074464983889, FiniteFieldSecp256k1::55134375882845171746885936987120838484249382119283764445714826914937932122379),
 FiniteFieldCyclicSecp256k1::8048113647876962713097111265089497531658399202432346479072980339276437701418)

In [365]:
list(zip(U, Pm[0]))[0]

(EllipticCurveGroupSecp256k1::(FiniteFieldSecp256k1::89873864151582986093691155183877263323500274185533681427305924574074464983889, FiniteFieldSecp256k1::55134375882845171746885936987120838484249382119283764445714826914937932122379),
 FiniteFieldCyclicSecp256k1::8048113647876962713097111265089497531658399202432346479072980339276437701418)

In [370]:
for i in zip(U, Pm[0]):
    print("u = %s, p = %s" % i)
    print(i[0]@i[1])
    print("----------")

u = (FiniteFieldSecp256k1::89873864151582986093691155183877263323500274185533681427305924574074464983889, FiniteFieldSecp256k1::55134375882845171746885936987120838484249382119283764445714826914937932122379), p = 8048113647876962713097111265089497531658399202432346479072980339276437701418
(FiniteFieldSecp256k1::65824645179792101789226153277123293846890079322289091195363093718873575643413, FiniteFieldSecp256k1::37900389520778853541876611911516817494261047007985620133540641301486635563099)
----------
u = (FiniteFieldSecp256k1::85722378055820376833749413035721572652460932312271018314254693083454536593925, FiniteFieldSecp256k1::96484742380690240068129470975691780944891565164112606247955288251377525188613), p = 255
(FiniteFieldSecp256k1::26864856020434440215424354932398950134176483136081872861807956664996689721835, FiniteFieldSecp256k1::103022691632056821848380530995373966494072348139789383049479942101527132449526)
----------
u = (FiniteFieldSecp256k1::433428418897546887970270170704094640591

TypeError: unsupported operand type(s) for &: 'FiniteFieldCyclicSecp256k1' and 'int'

In [379]:
G@CF(0)

TypeError: unsupported operand type(s) for &: 'FiniteFieldCyclicSecp256k1' and 'int'